In [2]:
import pandas as pd
import numpy as np

import json

In [82]:
_json = json.load(open("./eu_emission_latest_with_year.json"))
eu_brand_model = pd.json_normalize(_json['results'])
eu_brand_model

,Make,Fuel,Typ,Model,Cnt,Emission,year
0,RENAULT,Petrol,R,CLIO,401080,134.0,2018
1,CITROEN,petrol,S,C3,324540,135.0,2019
2,DACIA,petrol,SD,SANDERO,321984,146.0,2019
3,PEUGEOT,Petrol,C,208,315989,133.0,2018
4,DACIA,Petrol,SD,SANDERO,308575,147.0,2018
...,...,...,...,...,...,...,...
19995,MERCEDES-BENZ,diesel,R1ECLS,CLS 220 D,212,166.0,2019
19996,MERCEDES,petrol,245G,GLA 200 AMG LINE,212,170.0,2019
19997,BMW,DIESEL,G3X,X3 XDRIVE30D,212,NaN,2017
19998,MERCEDES BENZ,Diesel,218,CLASSE CLS,212,NaN,2012


In [168]:
eu_brand_model.sort_values('Emission')

,Make,Fuel,Model,Cnt,Emission,year
14554,NISSAN,ELECTRIC,E-NV200,392,0.0,2017
936,KIA,ELECTRIC,NIRO,17862,0.0,2019
10382,SKODA,ELECTRIC,ENYAQ 80,705,0.0,2020
4554,BMW,ELECTRIC,I3,2378,0.0,2017
8345,HYUNDAI,ELECTRIC,IONIQ PREMIUM SE EV,1000,0.0,2020
...,...,...,...,...,...,...
12163,JEEP,PETROL,GRAND CHEROKEE TRACKHAWK,540,382.0,2018
14855,BENTLEY,PETROL,FLYING SPUR V8 S,376,391.0,2018
17177,BENTLEY,PETROL,BENTAYGA V8,286,394.0,2018
13171,LAMBORGHINI,PETROL,AVENTADOR,470,487.0,2018


In [99]:

top_models

,Make,Fuel,Typ,Model,Cnt,Emission,year
0,RENAULT,PETROL,R,CLIO,401080,134.0,2018
1,CITROEN,PETROL,S,C3,324540,135.0,2019
2,DACIA,PETROL,SD,SANDERO,321984,146.0,2019
3,PEUGEOT,PETROL,C,208,315989,133.0,2018
4,DACIA,PETROL,SD,SANDERO,308575,147.0,2018
...,...,...,...,...,...,...,...
1503,BMW,DIESEL,F1X,X1 SDRIVE 18D,10026,147.0,2018
1505,MERCEDES-BENZ,DIESEL,639/2,VITO TOURER,10022,199.0,2020
1506,MERCEDES-BENZ,PETROL,245G,CLA 200,10014,172.0,2018
1507,TOYOTA,DIESEL,J15TM,TOYOTA LAND CRUISER (150 SERIES),10014,251.0,2018


## Data Cleaning

In [95]:
# json.dump({make:make  for make in eu_brand_model['Make'].unique()}, open("make_renaming.json", "w"), indent=4) 

In [158]:
json.dump({f:f  for f in eu_brand_model['Fuel'].unique()}, open("fuel_renaming.json", "w"), indent=4) 

In [93]:
eu_brand_model['Fuel'] = eu_brand_model['Fuel'].str.upper()
eu_brand_model['Fuel'] = eu_brand_model['Fuel'].str.strip()
eu_brand_model['Make'] = eu_brand_model['Make'].str.upper()
eu_brand_model['Make'] = eu_brand_model['Make'].str.strip()
eu_brand_model['Model'] = eu_brand_model['Model'].str.upper()
eu_brand_model['Model'] = eu_brand_model['Model'].str.strip()


In [94]:
make_renaming_table = json.load(open("make_renaming.json"))
eu_brand_model['Make'] = eu_brand_model['Make'].map(lambda s: make_renaming_table.get(s) or s)

In [155]:
eu_brand_model.drop('Typ', axis=1, inplace=True)
eu_brand_model.replace('', np.nan, inplace=True)
eu_brand_model.dropna(inplace=True)

## Brand Specific Cleaning

In [167]:
eu_brand_model.groupby(['Make']).Cnt.sum().sort_values(ascending=False)

Make
VOLKSWAGEN           7902925
RENAULT              5235661
PEUGEOT              5174652
FORD                 4389648
MERCEDES-BENZ        3786680
                      ...   
REULT                    540
DFSK                     478
MPM MOTORS               240
MORGAN                   218
DACIA - ROMTURING        212
Name: Cnt, Length: 65, dtype: int64

In [ ]:
top_models = eu_brand_model.loc[eu_brand_model.Cnt >= 5000].dropna()
top_models.loc[top_models['Make']=='AUDI'].Model.unique()

In [136]:
# AUDI
def audi(model: str):
    if model[0] in ['A', 'Q', 'S'] and model[:4] != "AUDI":
        return model.split(' ')[0]
    else:
        return model
    
top_models.loc[top_models['Make']=='AUDI'].Model.map(audi).unique()

array(['A4', 'A1', 'Q5', 'Q3', 'A6', 'Q2', 'A3', 'A5', 'Q8',
       'AUDI E-TRON', 'Q7', 'SQ5', 'E-TRON 50', 'SQ7', 'RS 3 SPORTBACK',
       'E-TRON 55', 'A7', 'S6', 'TT COUPE', 'RS3 SPORTBACK', 'E-TRON'],
      dtype=object)

In [139]:
def bmw(model: str):
    if model[0].isdigit() or model[1].isdigit():
        return model.split(' ')[0]
    else:
        return model

top_models.loc[top_models['Make']=='BMW'].Model.map(bmw).unique()

array(['118I', 'X3', 'X1', 'I3', 'X5', '116D', '520D', '320D', '225XE',
       '118D', '530E', 'SERIE X', '218I', '318D', '116I', '320I', 'I3S',
       '330E', 'X2', 'X4', '120I', '330I', '420D', '530D', 'SERIE 1',
       '420I', 'M2', '216D', '218D', 'M140I', 'Z4', '220I', '318I',
       '540I', 'M4', '120D', '520I'], dtype=object)

In [157]:
eu_brand_model.sort_values(['Make', 'Make', 'Fuel', 'year']).to_csv("eu_emission_latest_with_year.csv")

In [4]:
df = pd.read_csv("eu_emission_latest_with_year.csv")

In [36]:
make_model_count = df.groupby(['Make', 'Model']).Cnt.sum().sort_values(ascending=False)

In [43]:
make_model_count[make_model_count>10000].reset_index().sort_values(by=["Make", "Model"]).to_csv("make_model.csv", index=False)

In [31]:
make_model_count = df.sort_values(by="Cnt", ascending=False)

In [32]:
make_model_count.loc[make_model_count['Cnt']>10000].sort_values(['Make', 'Model', 'Cnt']).to_csv("make_model_count.csv")